In [1]:
"""
Building the patent data set by merging files from the USPTO.

Author: Joe Emmens

Final output is a dataset which records at the
    patent x inventor x assignee x IPC classification code
and records the following variables:
    1) num inventors
    2) citations
    3) year
"""

'\nBuilding the patent data set by merging files from the USPTO.\n\nAuthor: Joe Emmens\n'

In [2]:
import pandas as pd
import os
import numpy as np
import pickle
from datetime import datetime

In [3]:
os.chdir("D:\\IDEA Masters\\TFM Data\\IPC to Trade codes\\SITC_Rev2")

In [5]:
"""$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$

                                        Build the data set by merging the data from
                                                    USPTO downloads.

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$"""

"""
Load the inventor data which records one row for each inventor and their location on each patent.
Inventors location can vary since they live in different places and work remotely.
"""
inventor = pd.read_csv("patent_inventor.tsv", sep="\t", usecols=["patent_id", "inventor_id", "location_id"],
                       dtype={"patent_id":"str", "inventor_id":"str", "location_id":"str"})

In [7]:
"""
Load the same data but for the assignees, normally firms.
"""
assignee = pd.read_csv("patent_assignee.tsv", sep="\t", usecols=["patent_id", "assignee_id", "location_id"],
                       dtype={"patent_id":"str", "assignee_id":"str", "location_id":"str"})

6746031

In [23]:
inventor = inventor.merge(assignee, how="inner", on="patent_id")

del assignee

In [27]:
print(len(inventor["inventor_id"].unique()))
print(len(inventor["patent_id"].unique()))
len(inventor["assignee_id"].unique())

3219606
6745178


439937

In [26]:
"""
Load the classification data. Some patents are categorised under numerous tech
classes. The mean number is 1.45.
"""
ipc = pd.read_csv("ipcr1.tsv", sep="\t", low_memory=False, usecols=["patent_id", "section", "ipc_class", "subclass"],
                  dtype={"patent_id":"str", "section":"str", "ipc_class":"str", "sub_class":"str"})
ipc = ipc.drop_duplicates()
len(ipc["patent_id"].unique())

7114848

In [30]:
check = ipc.groupby("patent_id").count()

In [37]:
print(check.section.max())
print(check.section.mean())
print(check.section.median())

38
1.4566694889335654
1.0


In [38]:
idx = check.index[check["section"] > 1]
len(idx)

2054135

In [39]:
inventor = inventor.merge(ipc, how="inner", on="patent_id")

del ipc

print(len(inventor["inventor_id"].unique()))
print(len(inventor["patent_id"].unique()))
len(inventor["assignee_id"].unique())

3140504
6353519


412816

In [40]:
"""
Load the general patent data to get the year of award. There are numerous types of patents.
"""
parse_dates = ["date"]
patent = pd.read_csv("patent.tsv", sep="\t", usecols=["number", "date", "type"], parse_dates=parse_dates,
                     dtype={"number":"str", "date":"str", "type":"str"})

In [41]:
print(len(patent["number"].unique()))
print(len(patent[patent["type"] == "utility"]))

7627229
6913035


In [42]:
patent = patent.loc[patent["type"] == "utility"]
patent = patent.rename(columns={"number":"patent_id"})

In [44]:
inventor = inventor.merge(patent, how="inner", on="patent_id")

del patent
print(len(inventor["inventor_id"].unique()))
print(len(inventor["patent_id"].unique()))
len(inventor["assignee_id"].unique())

3113294
6209284


402407

In [47]:
"""
Leave the new word combinations for later as a robustness check if I think appropriate.

text_measures = pd.read_csv("patent_text_measures.txt", usecols=["patent", "new_word_comb", "new_word_comb_reuse"])
text_measures["patent"] = text_measures["patent"].astype("str")
text_measures = text_measures.rename(columns={"patent":"patent_id"})

print(len(text_measures["patent_id"].unique()))


inventor = inventor.merge(text_measures, how="inner", on="patent_id")
del text_measures
"""

In [49]:
"""
The final stats after merging the USPTO datasets are:
"""
print(len(inventor["inventor_id"].unique()))
print(len(inventor["patent_id"].unique()))
len(inventor["assignee_id"].unique())

3113294
6209284


402407

In [50]:
inventor["ipc_class"] = inventor["ipc_class"].astype("str")
inventor = inventor.drop(columns=["type"])

In [4]:
"""$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$

                                        Clean the data and add in variables of interest.
            1) The number of citations 2) Keep only the IPC codes which are currently in use

$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$"""



"""
Count the number of citations that each patent receives.
Be careful to keep the patents that receive 0 citations.
"""
inventor = pd.read_csv("Inventor Data.csv")

C:\ProgramData\Anaconda3\envs\pythonProject\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (1,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
print(len(inventor["inventor_id"].unique()))
print(len(inventor["patent_id"].unique()))
len(inventor["assignee_id"].unique())

3113294
6209316


402407

In [5]:
patents = pd.read_csv("uspatentcitation.tsv", sep="\t", usecols=["citation_id"],dtype={"citation_id":"str"})
patents["count"] = 1
check = patents.groupby("citation_id").count()

del patents
check = check.reset_index()

citations = dict(zip(check["citation_id"], check["count"]))
pickle.dump(citations, open("citations.p", "wb"))
del check

In [4]:
inventor = pd.read_csv("Inventor Data.csv")
citations = pickle.load(open("citations.p", "rb"))

inventor["patent_id"] = inventor["patent_id"].astype("str")
inventor["citations"] = inventor["patent_id"].map(citations)

inventor["citations"] = inventor["citations"].fillna(0)
inventor.to_csv("Inventor Data.csv", index_label=False)

C:\ProgramData\Anaconda3\envs\pythonProject\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (1,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
"""
Keep only the current IPC sections and classes.
    1) Drop all patents with other classifications. They aren't many...
    2) Ensure all single digit classes have a leading zero
"""

print(len(inventor["inventor_id"].unique()))
print(len(inventor["patent_id"].unique()))
len(inventor["assignee_id"].unique())


3113294
6209284


402407

In [6]:
inventor = inventor.loc[inventor["section"].isin(["A", "B", "C", "D", "E", "F", "G", "H"])]
print(len(inventor["inventor_id"].unique()))
print(len(inventor["patent_id"].unique()))
len(inventor["assignee_id"].unique())

3113088
6208546


402364

In [8]:
inventor["ipc_class"] = inventor["ipc_class"].astype("str")
inventor["section_class"] = inventor["section"] + inventor["ipc_class"]


class_dict = {
    "A1":"A01","B1":"B01","B2":"B02","B3":"B03","B4":"B04","B5":"B05","B6":"B06","B7":"B07",
    "B8":"B08","B9":"B09","C1":"C01","C2":"C02","C3":"C03","C4":"C04",
    "C5":"C05","C6":"C06","C7":"C07","C8":"C08","C9":"C09","D1":"D01", "D2":"D02","D3":"D03","D4":"D04",
    "D5":"D05","D6":"D06","D7":"D07","E1":"E01","E2":"E02","E3":"E03",
    "E4":"E04","E5":"E05","E6":"E06","F1":"F01","F2":"F02","F3":"F03","F4":"F04",
    "G1":"G01", "G2":"G02","G3":"G03","G4":"G04","G5":"G05","G6":"G06","G7":"G07",
    "G8":"G08","G9":"G09","H1":"H01","H2":"H02","H3":"H03","H4":"H04","H5":"H05",
}

inventor["section_class"] = inventor["section_class"].replace(class_dict)

inventor["section_class_sub"] = inventor["section_class"] + inventor["subclass"]

inventor = inventor.drop(columns=["ipc_class", "subclass"])

In [14]:
check = inventor["section_class"].value_counts()

In [15]:
inventor.to_csv("Inventor Data.csv", index_label=False)

In [17]:
"""
Keep only the classifications that are on the product space using the official IPC codes.
"""
os.chdir("D:\\IDEA Masters\\TFM Data\\IPC to Trade codes\\SITC_Rev2")
ipc_subclass = pd.read_csv("ipc4_to_sitc_rev2_4.txt")
official_codes_sub = list(ipc_subclass.ipc4.unique())

inventor = inventor[inventor["section_class_sub"].isin(official_codes_sub)]

print(len(inventor["inventor_id"].unique()))
print(len(inventor["patent_id"].unique()))
len(inventor["assignee_id"].unique())

3108197
6190712


401411

In [22]:
"""
Strip the date so only displays the year and drop the date
"""
inventor["year"] = inventor["date"].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').year)

inventor = inventor.drop(columns="date")

In [30]:
"""
Check whether the number of IPC codes match. They are very very close.
"""
check2 = inventor["section_class"].value_counts()
check1 = inventor["section_class_sub"].value_counts()

In [32]:
print(len(check1.index))
print(len(check2.index))

634
121


In [34]:

no_inv = inventor.groupby(["patent_id", "section_class_sub"])["inventor_id"].count()
no_inv = no_inv.reset_index().drop(columns="section_class_sub")
no_inv = no_inv.drop_duplicates()
no_inv = no_inv.rename(columns={"inventor_id":"no_invs"})
no_inv_dict = dict(zip(no_inv.patent_id, no_inv.no_invs))
inventor["no_inventors"] = inventor.patent_id.map(no_inv_dict)

In [9]:
print(np.mean(inventor["no_inventors"]))
print(np.max(inventor["no_inventors"]))

4.999169463388831
374


In [8]:
inventor.to_csv("Inventor Data.csv", index_label=False)

In [ ]:
"""
Save the final dataset
"""
inventor.to_csv("Inventor Data.csv", index_label=False)


